In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install japanize-matplotlib

     |████████████████████████████████| 4.1 MB 27.4 MB/s 
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120275 sha256=ea08f73501c258ab6929665f5036decf4b95952aed83978b084722a33bb2377a
  Stored in directory: /root/.cache/pip/wheels/83/97/6b/e9e0cde099cc40f972b8dd23367308f7705ae06cd6d4714658
Successfully built japanize-matplotlib


In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np
import japanize_matplotlib

In [ ]:
current_dir=os.getcwd()
current_dir

'/content'

In [ ]:
cd /content/drive/MyDrive/input/

/content/drive/MyDrive/input


In [ ]:
os.listdir(current_dir)

['.config', 'drive', 'sample_data']

In [ ]:
ls

holidays_events.csv  sample_submission.csv  train.csv      transactions.csv
oil.csv              stores.csv             train_df1.csv


In [ ]:
#読み込み
holidays=pd.read_csv("holidays_events.csv")
oil = pd.read_csv("oil.csv")
stores = pd.read_csv("stores.csv")
transactions = pd.read_csv("transactions.csv")
train = pd.read_csv("train.csv")

In [ ]:
#マージ
train_oil=train.merge(oil,how='left',on='date')
train_oil_holidays=train_oil.merge(holidays,how='left',on='date')
train_oil_holidays_transactions=train_oil_holidays.merge(transactions,how='left',on=['date','store_nbr'])
train_oil_holidays_transactions_stores=train_oil_holidays_transactions.merge(stores,how='left',on=['type','store_nbr']).sort_values(by=['date']).reset_index(drop=True)

In [ ]:
#マージしたものをtrainとする
train=train_oil_holidays_transactions.copy()

In [ ]:
#dateの項目から、Day,Week,Month,Year,Weekdayの項目を追加
train['date'] = pd.to_datetime(train['date'])
#df[date]object     ->      datetime
print(train['date'].dtypes)
#項目追加
train['Day'] = train['date'].dt.day
train['Week'] = train['date'].dt.week
train['Month'] = train['date'].dt.month
train['Year'] = train['date'].dt.year
train['Weekday']=train['date'].dt.weekday

#Dayは外してもいいかも

datetime64[ns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  import sys


In [ ]:
pd.DataFrame(train.dtypes, columns=['Type']).T

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,type,locale,locale_name,description,transferred,transactions,Day,Week,Month,Year,Weekday
Type,int64,datetime64[ns],int64,object,float64,int64,float64,object,object,object,object,object,float64,int64,int64,int64,int64,int64


In [ ]:
a=train['type'].unique()
print('type=',a)
print('')
b=train['locale'].unique()
print('locale=',b)
print('')
c=train['locale_name'].unique()
print('locale_name=',c)
print('')
d=train['description'].unique()	
print('description=',d)
print('')
print('transferred:転送されたカラムには特に注意してください。正式に譲渡された休日はその暦日になりますが、政府によって別の日付に移動されました。転送された日は、休日というよりも通常の日に似ています。実際に祝われた日を見つけるには、タイプが転送である対応する行を探します。')
e=train['transferred'].unique()
print('transferred=',e)
print('')
g=train['type'].unique()
print('type=',g)
print('')

h=train['family'].unique()
print('family=',h)
print('')

type= ['Holiday' nan 'Work Day' 'Additional' 'Event' 'Transfer' 'Bridge']

locale= ['National' nan 'Local' 'Regional']

locale_name= ['Ecuador' nan 'Manta' 'Cotopaxi' 'Cuenca' 'Libertad' 'Riobamba' 'Puyo'
 'Guaranda' 'Imbabura' 'Machala' 'Latacunga' 'El Carmen' 'Santo Domingo'
 'Cayambe' 'Guayaquil' 'Esmeraldas' 'Ambato' 'Ibarra' 'Quevedo'
 'Santo Domingo de los Tsachilas' 'Santa Elena' 'Quito' 'Loja' 'Salinas']

description= ['Primer dia del ano' nan 'Recupero puente Navidad'
 'Recupero puente primer dia del ano' 'Carnaval' 'Fundacion de Manta'
 'Provincializacion de Cotopaxi' 'Fundacion de Cuenca'
 'Cantonizacion de Libertad' 'Cantonizacion de Riobamba' 'Viernes Santo'
 'Dia del Trabajo' 'Dia de la Madre-1' 'Cantonizacion del Puyo'
 'Dia de la Madre' 'Batalla de Pichincha' 'Cantonizacion de Guaranda'
 'Provincializacion de Imbabura' 'Fundacion de Machala'
 'Cantonizacion de Latacunga' 'Cantonizacion de El Carmen'
 'Fundacion de Santo Domingo' 'Cantonizacion de Cayambe'
 'Fundacion de

In [ ]:
# train['type']=train['type'].replace(['Work Day'],['5'])

In [ ]:
train=train.fillna(0)

In [ ]:
train.head()

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,type,locale,locale_name,description,transferred,transactions,Day,Week,Month,Year,Weekday
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,0.0,Holiday,National,Ecuador,Primer dia del ano,False,0.0,1,1,1,2013,1
1,1,2013-01-01,1,BABY CARE,0.0,0,0.0,Holiday,National,Ecuador,Primer dia del ano,False,0.0,1,1,1,2013,1
2,2,2013-01-01,1,BEAUTY,0.0,0,0.0,Holiday,National,Ecuador,Primer dia del ano,False,0.0,1,1,1,2013,1
3,3,2013-01-01,1,BEVERAGES,0.0,0,0.0,Holiday,National,Ecuador,Primer dia del ano,False,0.0,1,1,1,2013,1
4,4,2013-01-01,1,BOOKS,0.0,0,0.0,Holiday,National,Ecuador,Primer dia del ano,False,0.0,1,1,1,2013,1


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train['type'] = le.fit_transform(train['type'].astype(str))
train['locale'] = le.fit_transform(train['locale'].astype(str))
train['description'] = le.fit_transform(train['description'].astype(str))
train['transferred'] = le.fit_transform(train['transferred'].astype(int))
train['locale_name'] = le.fit_transform(train['locale_name'].astype(str))
train['family'] = le.fit_transform(train['family'].astype(str))

In [ ]:
train=train.sort_values('date')

In [ ]:
train.head(5)

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,type,locale,locale_name,description,transferred,transactions,Day,Week,Month,Year,Weekday
0,0,2013-01-01,1,0,0.0,0,0.0,4,2,5,51,0,0.0,1,1,1,2013,1
1194,1194,2013-01-01,42,6,0.0,0,0.0,4,2,5,51,0,0.0,1,1,1,2013,1
1193,1193,2013-01-01,42,5,0.0,0,0.0,4,2,5,51,0,0.0,1,1,1,2013,1
1192,1192,2013-01-01,42,4,0.0,0,0.0,4,2,5,51,0,0.0,1,1,1,2013,1
1191,1191,2013-01-01,42,3,0.0,0,0.0,4,2,5,51,0,0.0,1,1,1,2013,1


In [ ]:
!pip install sweetviz -q

     |████████████████████████████████| 15.1 MB 28.8 MB/s 


In [ ]:
import sweetviz
# sweetvizを使って学習用データのサマリーを確認
report_train = sweetviz.analyze(train, # データフレーム 
                                target_feat='sales') # 目的変数の列名
report_train.show_notebook() # サマリーを表示

In [ ]:
train.to_csv("store_sales.csv", index=False )